In [1]:
"""Postpartum Depression Classification with BETR/PsghGPT2/T5"""
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# === Parameters ===
MODEL_NAME = 'roberta-large'
NUM_LABELS = 3
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 10
TFIDF_DIM = 1000
LR = 1e-5

# === Load Data ===
df = pd.read_csv("ppd_data.csv")

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True
    )

# === TF-IDF Features ===
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=TFIDF_DIM)
tfidf_features = tfidf_vectorizer.fit_transform(df['text']).toarray()

# === Tokenization ===
input_ids, attention_masks = [], []
for text in df['text']:
    encoded = tokenize_text(text)
    input_ids.append(encoded['input_ids'].squeeze(0))
    attention_masks.append(encoded['attention_mask'].squeeze(0))

# === Dataset ===
class FusionDataset(Dataset):
    def __init__(self, input_ids, attention_masks, tfidf_features, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.tfidf_features = tfidf_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'tfidf': torch.tensor(self.tfidf_features[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# === Train/Val Split ===
train_idx, temp_idx = train_test_split(np.arange(len(df)), test_size=0.2, stratify=df['label'])
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=df.loc[temp_idx, 'label'])


train_dataset = FusionDataset(
    [input_ids[i] for i in train_idx],
    [attention_masks[i] for i in train_idx],
    tfidf_features[train_idx],
    df['label'].values[train_idx]
)

val_dataset = FusionDataset(
    [input_ids[i] for i in val_idx],
    [attention_masks[i] for i in val_idx],
    tfidf_features[val_idx],
    df['label'].values[val_idx]
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# === Model ===
class BertWithTFIDF(nn.Module):
    def __init__(self, model_name, tfidf_dim, num_labels):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(hidden_size + tfidf_dim, num_labels)

    def forward(self, input_ids, attention_mask, tfidf_features):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        fused = torch.cat((cls_output, tfidf_features), dim=1)
        fused = self.dropout(fused)
        return self.classifier(fused)

# === Train ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertWithTFIDF(MODEL_NAME, TFIDF_DIM, NUM_LABELS).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        tfidf = batch['tfidf'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask, tfidf)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Train Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_preds, val_labels, val_loss = [], [], 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            tfidf = batch['tfidf'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask, tfidf)
            loss = loss_fn(logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(val_labels, val_preds)
    f1 = f1_score(val_labels, val_preds, average='weighted')
    print(f"[Epoch {epoch+1}] Val Loss: {val_loss / len(val_loader):.4f} | Acc: {acc:.4f} | Weighted F1: {f1:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Epoch 1] Train Loss: 1.0475
[Epoch 1] Val Loss: 1.0449 | Acc: 0.3854 | Weighted F1: 0.2855
[Epoch 2] Train Loss: 0.9141
[Epoch 2] Val Loss: 0.7792 | Acc: 0.6146 | Weighted F1: 0.5746
[Epoch 3] Train Loss: 0.7618
[Epoch 3] Val Loss: 0.7671 | Acc: 0.5938 | Weighted F1: 0.5762
[Epoch 4] Train Loss: 0.6588
[Epoch 4] Val Loss: 0.8443 | Acc: 0.5833 | Weighted F1: 0.5899
[Epoch 5] Train Loss: 0.4999
[Epoch 5] Val Loss: 0.7928 | Acc: 0.6771 | Weighted F1: 0.6782
[Epoch 6] Train Loss: 0.3349
[Epoch 6] Val Loss: 0.8830 | Acc: 0.6771 | Weighted F1: 0.6671
[Epoch 7] Train Loss: 0.2492
[Epoch 7] Val Loss: 0.9094 | Acc: 0.6771 | Weighted F1: 0.6759
[Epoch 8] Train Loss: 0.1520
[Epoch 8] Val Loss: 1.0375 | Acc: 0.6771 | Weighted F1: 0.6603
[Epoch 9] Train Loss: 0.0840
[Epoch 9] Val Loss: 0.9928 | Acc: 0.7083 | Weighted F1: 0.7035
[Epoch 10] Train Loss: 0.0386
[Epoch 10] Val Loss: 1.0139 | Acc: 0.7292 | Weighted F1: 0.7339


In [5]:
# === Tokenization ===
input_ids, attention_masks = [], []
for text in df['text']:
    encoded = tokenize_text(text)
    input_ids.append(encoded['input_ids'].squeeze(0))
    attention_masks.append(encoded['attention_mask'].squeeze(0))

test_dataset = FusionDataset(
    [input_ids[i] for i in test_idx],
    [attention_masks[i] for i in test_idx],
    tfidf_features[test_idx],
    df['label'].values[test_idx]
)


test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
model.eval()
test_preds, test_labels, test_loss = [], [], 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        tfidf = batch['tfidf'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask, tfidf)
        loss = loss_fn(logits, labels)
        test_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
print(f"[Test] Loss: {test_loss / len(test_loader):.4f} | Acc: {test_acc:.4f} | Weighted F1: {test_f1:.4f}")


[Test] Loss: 1.4730 | Acc: 0.6667 | Weighted F1: 0.6744


In [10]:
def compute_metrics(preds, true_labels):
    return {
        "accuracy": accuracy_score(true_labels, preds),
        "f1_weighted": f1_score(true_labels, preds, average="weighted"),
        "f1_macro": f1_score(true_labels, preds, average="macro")
    }
metrics = compute_metrics(test_preds, test_labels)

print(f"[Test Metrics]")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"F1 (Weighted): {metrics['f1_weighted']:.4f}")
print(f"F1 (Macro): {metrics['f1_macro']:.4f}")


[Test Metrics]
Accuracy: 0.6667
F1 (Weighted): 0.6744
F1 (Macro): 0.6265


In [6]:
def predict_rating(text):
    model.eval()
    with torch.no_grad():
        # Tokenizer
        encoded = tokenizer.encode_plus(
            text,
            max_length=MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        input_ids = encoded['input_ids'].to(device)
        attention_mask = encoded['attention_mask'].to(device)

        # TF-IDF
        tfidf_feat = tfidf_vectorizer.transform([text]).toarray()
        tfidf_tensor = torch.tensor(tfidf_feat, dtype=torch.float).to(device)

        # Predict
        logits = model(input_ids, attention_mask, tfidf_tensor)
        pred = torch.argmax(logits, dim=1).item()
        return f"Predicted Rating: {pred}"
